## How Pre-existing Socio-economic Factors of U.S.Counties Relate to the Spread of COVID-19

### Research Objectives
    
    RQ1: Is there are correlation between pre-existing socio-economic factors and COVID-19 cases, and which factors have the highest correlation?
    RQ2: Do socio-economic factors differ by state and how might this information aid state and local government responses?

### External Data Sources
    
    COVID-19 cases, and deaths obtained from:
        https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/
        
    Employment, Education, Poverty data obtained from:
        https://www.ers.usda.gov/data-products/county-level-data-sets/
    
    County population by sex obtained from:
        https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html
    
    File names have been changed for easier typing and referencing - no changes have been to the data itself.

### Socio-economic factors
    
    Focus is kept only on the following factors. There is high correlation with other factors like number of ICU beds but it is not a socio-economic factor so it is ignored altogether. Also note there is a model that is able to explain 97% of the data in the U.S. but it does not pertain to socio-economic factors so it is not studied, however it is shown below for reference/knowledge gathering purposes.
    
    Age
    Race
    Income
    Family Size
    Employement
    Education
    Gender
    Poverty Levels
    Housing
    
    References: 
    https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-factors/social-and-economic-factors
    https://en.wikipedia.org/wiki/Socioeconomic_status

In [1]:
import pandas as pd
import numpy as np
import math 

from rpy2.robjects import pandas2ri
pandas2ri.activate()
%load_ext rpy2.ipython

from IPython.display import Markdown, display

In [2]:
# reference https://stackoverflow.com/questions/54586615/pandas-modify-column-values-based-on-dictionary-values
us_state_abbrev = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
                   'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
                   'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
                   'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
                   'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
                   'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
                   'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
                   'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
                   'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'}

In [3]:
# load data
county_data = pd.read_csv("JHU_COUNTY_DATA.csv")
covid_cases = pd.read_csv("COVID_CASES.csv")
covid_deaths = pd.read_csv("COVID_DEATHS.csv")
poverty = pd.read_excel("POVERTY_ESTIMATES.xls", skiprows=4)
unemployment = pd.read_excel("UNEMPLOYMENT.xls", skiprows=7)
education = pd.read_excel("EDUCATION.xls", skiprows=4)
population = pd.read_csv("COUNTY_SEX_DATA.csv", encoding="ISO-8859-1")
population = population.loc[(population.YEAR == 11) & (population.AGEGRP == 0)]
population.reset_index(inplace=True, drop=True)

Check how many rows we have in each of the datasets above.

In [4]:
print(f"Number of rows in county_data: {len(county_data)}")
print(f"Number of rows in covid_cases: {len(covid_cases)}")
print(f"Number of rows in covid_deaths: {len(covid_deaths)}")
print(f"Number of rows in population: {len(population)}")
print(f"Number of rows in poverty: {len(poverty)}")
print(f"Number of rows in unemployment: {len(unemployment)}")
print(f"Number of rows in education: {len(education)}")

Number of rows in county_data: 3138
Number of rows in covid_cases: 3195
Number of rows in covid_deaths: 3195
Number of rows in population: 3142
Number of rows in poverty: 3193
Number of rows in unemployment: 3275
Number of rows in education: 3283


#### County Data

In [5]:
def impute_missing_information(df, states):
    # from the states dictionary get 
    # state abbrivations
    states = states.values()
    # for each state in states list
    for state in states:
        # find if there any missing values and
        # create a dictionary of number of values
        # missing in a feature
        info = df.loc[df["STATE_ABBR"] == state].isna().sum().to_dict()
        # for every feature in info variable
        for key in info:
            # if the value is greater than 0
            # this means there are missing 
            # values in the feature
            num = info[key]
            if num > 0:
                # impute mean of the column for the state 
                impute = df.loc[df["STATE_ABBR"] == state][key].mean()
                index = df[key].index[df[key].apply(np.isnan)]
                df.at[index, key] = np.round(impute, 0)
    return df

Above function checks for missing values in county_data. If there is a missing value then the state for which the value is missing is selected, and then mean of column where the value is missing is imputed for the missing value. This way we can guarantee that if we do not have unemployment data for Nebraska for instance, then we are only imputing mean unemployment for Nebraska and not the whole country.

In [6]:
def add_county(x):
    if x.endswith("City"):
        return x
    if x == "District of Columbia":
        return x
    else:
        return x + " County"

def county_state(x):
    return x[0] + ", " + x[1]

Some of the other data files that have gathered from various sources have "County" added at then end of the county name. The functions above allow for matching of information. Second function, county_state, simply adds the state abbrevation to the end of the county name. For instance, one of the counties in JHU_COUNTY is Autauga, after the first function we have COUNTY feature with Autauga County. Second function assigns the state of the county, so we have Autauga County, AL. Some counties name repeat for instance there is Baldwin County in AL aswell as in GA, attention is paid to repeating county names. These names are correctly assigned because FIPS is unique to a county so state abbrevation is added to a county based on FIPS.

In [7]:
county_data = impute_missing_information(county_data, us_state_abbrev)

In [8]:
county_data["COUNTY"] = county_data["JHU_COUNTY"].apply(lambda x: add_county(x))
county_data["COUNTY_STATE"] = county_data["JHU_FIPS"].apply(lambda x: county_state(county_data.loc[county_data["JHU_FIPS"] == x][["COUNTY", "STATE_ABBR"]].values[0]))

In [9]:
county_data.drop(columns=["STATE_COUNTY", "STATE_COUNTY_LC", "JHU_COMB_KEY",
                         "GEOGRAPHY", "JHU_COUNTRY", "STATE_LC"], inplace=True)

county_data.reset_index(inplace=True, drop=True)

In [10]:
display(Markdown("**COUNTY DATAFRAME**"))
county_data.head(3)

**COUNTY DATAFRAME**

,JHU_FIPS,JHU_PROV_STATE,JHU_COUNTY,STATE_ABBR,COUNTY_ABBR,POPEST_2018_B_TOTAL,POPEST_2018_B_U18,POPEST_2018_B_18TO64,POPEST_2018_B_65PLUS,POPEST_2018_B_85PLUS,...,INCOME_RATIO,PER_UNEMPLOYED,PER_BLACK,PER_NATIVE,PER_ASIAN,PER_PACIFIC,PER_HIS,PER_WHITE,COUNTY,COUNTY_STATE
0,1001,Alabama,Autauga,AL,"Autauga,AL",55601,13163,33785,8653,871,...,5.234597,0.027882,0.193432,0.004802,0.012248,0.001115,0.029658,0.743080,Autauga County,"Autauga County, AL"
1,1003,Alabama,Baldwin,AL,"Baldwin,AL",218022,47110,126341,44571,4304,...,4.417767,0.026856,0.087840,0.007724,0.011503,0.000670,0.046468,0.831113,Baldwin County,"Baldwin County, AL"
2,1005,Alabama,Barbour,AL,"Barbour,AL",24881,5192,14857,4832,472,...,5.681410,0.029145,0.480326,0.006591,0.004542,0.001849,0.042764,0.456413,Barbour County,"Barbour County, AL"


The very last column, COUNTY_STATE, is what is created.

In [11]:
COUNTIES = list(county_data["COUNTY_STATE"].values)
FIPS = list(county_data["JHU_FIPS"].values)

#### COVID-19 Cases

In [12]:
covid_cases["FIPS"] = covid_cases["countyFIPS"]
covid_cases["COUNTY_STATE"] = covid_cases["countyFIPS"].apply(lambda x: county_state(covid_cases.loc[covid_cases["countyFIPS"] == x][["County Name", "State"]].values[0]))

In [13]:
covid_cases["sum"] = covid_cases["12/9/20"]

Cases data reflects cumulative cases for each date. So on 12/9/20 cases reported are the total number of cases reported for the county since the tracking started.

In [14]:
display(Markdown("**COVID-19 CASES BY COUNTY BY DATE**"))
covid_cases.head(3)

**COVID-19 CASES BY COUNTY BY DATE**

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,FIPS,COUNTY_STATE,sum
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"Statewide Unallocated, AL",0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,2893,2945,2979,3005,3043,3087,3117,1001,"Autauga County, AL",3117
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,9341,9501,9626,9728,9821,9974,10087,1003,"Baldwin County, AL",10087


In [15]:
total_cases = covid_cases["sum"].sum()
print(f"Total Number of COVID-19 Cases in the U.S. as at December 09, 2020: {total_cases}")

Total Number of COVID-19 Cases in the U.S. as at December 09, 2020: 15181241


This information makes sense is backed by various credible sources like the JHU dashboard.

#### COVID-19 Deaths

This data is from the same soure as COVID-19 Cases data and follows the same format as above.

In [16]:
covid_deaths["FIPS"] = covid_deaths["countyFIPS"]
covid_deaths["COUNTY_STATE"] = covid_deaths["countyFIPS"].apply(lambda x: county_state(covid_deaths.loc[covid_deaths["countyFIPS"] == x][["County Name", "State"]].values[0]))

In [17]:
covid_deaths["sum"] = covid_deaths["12/9/20"]

In [18]:
display(Markdown("**COVID-19 DEATHS BY COUNTY BY DATE**"))
covid_deaths.head(3)

**COVID-19 DEATHS BY COUNTY BY DATE**

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,FIPS,COUNTY_STATE,sum
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"Statewide Unallocated, AL",0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,42,42,42,42,42,42,42,1001,"Autauga County, AL",42
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,137,138,138,138,138,141,141,1003,"Baldwin County, AL",141


In [19]:
total_deaths = covid_deaths["sum"].sum()
print(f"Total Number of COVID-19 Deaths in the U.S. as at December 09, 2020: {total_deaths}")

Total Number of COVID-19 Deaths in the U.S. as at December 09, 2020: 285043


This seems reasonable and makes sense based on available information.

#### Poverty Data

In [20]:
poverty["FIPS"] = poverty["FIPStxt"]
poverty["COUNTY_STATE"] = poverty["FIPStxt"].apply(lambda x: county_state(poverty.loc[poverty["FIPStxt"] == x][["Area_name", "Stabr"]].values[0]))

**NOTE:** The lambda function will create the following COUNTY_STATE information **United States, US** and **Alabama, AL** but these are of no consequence because we are only interested in the county level data. 

In [21]:
display(Markdown("**COUNTY POVERTY DATA**"))
poverty.head(3)

**COUNTY POVERTY DATA**

,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,...,CI90LBINC_2018,CI90UBINC_2018,POV04_2018,CI90LB04_2018,CI90UB04_2018,PCTPOV04_2018,CI90LB04P_2018,CI90UB04P_2018,FIPS,COUNTY_STATE
0,0,US,United States,NaN,NaN,NaN,NaN,41852315,41619366,42085264,...,61843,62031,3758704.0,3714862.0,3802546.0,19.5,19.3,19.7,0,"United States, US"
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,801758,785668,817848,...,49123,50639,73915.0,69990.0,77840.0,26.0,24.6,27.4,1000,"Alabama, AL"
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,7587,6334,8840,...,53628,65048,NaN,NaN,NaN,NaN,NaN,NaN,1001,"Autauga County, AL"


There are 36 columns, but we are only interested in information from 2018. In particular POVALL_2018 is the feature extracted from this dataset.

#### Unemployment Data

In [22]:
unemployment["FIPS"] = unemployment["FIPStxt"]
unemployment["COUNTY_STATE"] = unemployment["area_name"]

In [23]:
display(Markdown("**COUNTY UNEMPLOYMENT DATA**"))
unemployment.head(3)

**COUNTY UNEMPLOYMENT DATA**

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,FIPS,COUNTY_STATE
0,0,US,United States,NaN,NaN,NaN,142601667.0,136904680.0,5696987.0,3.995035,...,6286707.0,3.895375,163100055.0,157115247.0,5984808.0,3.669409,61937.0,NaN,0,United States
1,1000,AL,Alabama,NaN,NaN,NaN,2133223.0,2035594.0,97629.0,4.600000,...,85782.0,3.900000,2241747.0,2174483.0,67264.0,3.000000,49881.0,100.000000,1000,Alabama
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,21720.0,20846.0,874.0,4.000000,...,935.0,3.600000,26172.0,25458.0,714.0,2.700000,59338.0,118.959123,1001,"Autauga County, AL"


This data provides Median_Household_Income_2018, and Unemployed_2018.

#### Education Data

In [24]:
education["FIPS"] = education["FIPS Code"]
education["COUNTY_STATE"] = education["FIPS Code"].apply(lambda x: county_state(education.loc[education["FIPS Code"] == x][["Area name", "State"]].values[0]))

In [25]:
display(Markdown("**COUNTY EDUCATION DATA**"))
education.head(3)

**COUNTY EDUCATION DATA**

,FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Less than a high school diploma, 2014-18","High school diploma only, 2014-18","Some college or associate's degree, 2014-18","Bachelor's degree or higher, 2014-18","Percent of adults with less than a high school diploma, 2014-18","Percent of adults with a high school diploma only, 2014-18","Percent of adults completing some college or associate's degree, 2014-18","Percent of adults with a bachelor's degree or higher, 2014-18",FIPS,COUNTY_STATE
0,0,US,United States,NaN,NaN,NaN,NaN,52373312.0,34158051.0,11650730.0,...,26948057.0,59265308.0,63365655.0,68867051.0,12.3,27.1,29.0,31.5,0,"United States, US"
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,1062306.0,468269.0,136287.0,...,470043.0,1020172.0,987148.0,822595.0,14.2,30.9,29.9,24.9,1000,"Alabama, AL"
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,6611.0,3757.0,933.0,...,4204.0,12119.0,10552.0,10291.0,11.3,32.6,28.4,27.7,1001,"Autauga County, AL"


From this data we extract education levels. In particular: 

    Less than a high school diploma, 2014-18
    High school diploma only, 2014-18
    Bachelor's degree or higher, 2014-18

#### Population Data

We created FIPS, and COUNTY_STATE features in above datasets for easy referencing and to make sure that the name carries throughout the datasets. However, for this dataset we do not have FIPS column. In order to ensure that this dataset follows the same format following steps were taken:

In [26]:
dict(zip(county_data["COUNTY_STATE"],county_data["JHU_FIPS"]))
pd.Series(county_data["COUNTY_STATE"].values,index=county_data["JHU_FIPS"]).to_dict()
FIPS_DICT = county_data.set_index('COUNTY_STATE').to_dict()["JHU_FIPS"]

In [27]:
population["STATE_ABBR"] = population["STNAME"].apply(lambda x: us_state_abbrev[x])

In [28]:
def add_county_state_to_population_data(df):
    df["COUNTY_STATE"] = ""
    
    for idx, rows in df.iterrows():
        state = rows["STATE_ABBR"]
        county = rows["CTYNAME"]
        df.at[idx, "COUNTY_STATE"] = county + ", " + state
    
    return df

In [29]:
population = add_county_state_to_population_data(population)

In [30]:
population["FIPS"] = population["COUNTY_STATE"].apply(lambda x: FIPS_DICT[x] if x in FIPS_DICT.keys() else np.nan)

In [31]:
population.head(3)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,STATE_ABBR,COUNTY_STATE,FIPS
0,50,1,1,Alabama,Autauga County,11,0,55533,27006,28527,...,75,39,32,20,16,19,13,AL,"Autauga County, AL",1001.0
1,50,1,3,Alabama,Baldwin County,11,0,217855,105702,112153,...,263,258,186,62,56,50,31,AL,"Baldwin County, AL",1003.0
2,50,1,5,Alabama,Barbour County,11,0,24872,13148,11724,...,47,63,25,3,1,13,8,AL,"Barbour County, AL",1005.0


In [32]:
def missing(COUNTIES, covid_cases, covid_deaths, population, unemployment, education, poverty):
    error = {"CASES": [], "DEATHS": [], "POP": [], "UNEMP": [], "EDU": [], "POV": []}
    for county in COUNTIES:
        if len(covid_cases.loc[covid_cases["COUNTY_STATE"] == county]["sum"].values) == 0:
            error["CASES"].append(county)
        if len(covid_deaths.loc[covid_deaths["COUNTY_STATE"] == county]["sum"].values) == 0:
            error["DEATHS"].append(county)
        if len(population.loc[population["COUNTY_STATE"] == county]["TOT_POP"].values) == 0:
            error["POP"].append(county)
        if len(unemployment.loc[unemployment["COUNTY_STATE"] == county]["Median_Household_Income_2018"].values) == 0:
            error["UNEMP"].append(county)
        if len(education.loc[education["COUNTY_STATE"] == county]["Less than a high school diploma, 2014-18"].values) == 0:
            error["EDU"].append(county)
        if len(poverty.loc[poverty["COUNTY_STATE"] == county]["POVALL_2018"].values) == 0:
            error["POV"].append(county)
    union = list(set().union(error["CASES"], error["DEATHS"], error["POP"], error["UNEMP"], error["EDU"], error["POV"]))
    return union

It became evident above that not all the datasets have same number of examples. Therefore there must be some mismatch. Above function takes care of this. We get a list of counties that we in county_data and try to find them in other datasets. If a county does not exit, we simply put in a dictionary, with the name of dataset serving as the key. Next, we utilize set union to ensure we get a list of all elements that are missing in one or all of the datasets. 

In [33]:
missing = missing(COUNTIES, covid_cases, covid_deaths, population, unemployment, education, poverty)
print(f"Number of Counties for which information is missing: {len(missing)}")

Number of Counties for which information is missing: 151


Since the data does not have information for 151 counties, these counties will dropped from analysis from the county_data dataframe.

In [34]:
def add_features(county_data, missing, covid_cases, covid_deaths, population, unemployment, education, poverty):
    
    # less than high school diploma
    county_data["LESS_THAN_HSD"] = np.nan
    # only high school diploma
    county_data["ONLY_HSD"] = np.nan
    # bachelors or higher
    county_data["BCH_OR_HIGHER"] = np.nan
    # median income of state total
    county_data["MED_INC_STATE_TOTAL"] = np.nan
    # number of people living in poverty
    county_data["POVALL_2018"] = np.nan
    # total number of male 
    county_data["TOT_MALE"] = np.nan
    # total number of female
    county_data["TOT_FEMALE"] = np.nan
    # total population
    county_data["TOT_POP"] = np.nan
    # number of covid cases
    county_data["CASES"] = np.nan
    # average cases per day
    county_data["AVG_CASES_PER_DAY"] = np.nan
    # number of deaths per day
    county_data["DEATHS"] = np.nan
    # death rate
    county_data["DEATH_RATE"] = np.nan
    
    for idx, row in county_data.iterrows():
        county = row["COUNTY_STATE"]
        if county not in missing:
            
            num_cases = covid_cases.loc[covid_cases["COUNTY_STATE"] == county]["sum"].item()
            num_deaths = covid_deaths.loc[covid_deaths["COUNTY_STATE"] == county]["sum"].item()
            tot_pop = population.loc[population["COUNTY_STATE"] == county]["TOT_POP"].item()
            male_pop = population.loc[population["COUNTY_STATE"] == county]["TOT_MALE"].item()
            fem_pop = population.loc[population["COUNTY_STATE"] == county]["TOT_FEMALE"].item()
            poor = poverty.loc[poverty["COUNTY_STATE"] == county]["POVALL_2018"].item()
            med = unemployment.loc[unemployment["COUNTY_STATE"] == county]["Med_HH_Income_Percent_of_State_Total_2018"].item()
            less_than_hsd = education.loc[education["COUNTY_STATE"] == county]["Less than a high school diploma, 2014-18"].item()
            only_hsd = education.loc[education["COUNTY_STATE"] == county]["High school diploma only, 2014-18"].item()
            bch_or_higher = education.loc[education["COUNTY_STATE"] == county]["Bachelor's degree or higher, 2014-18"].item()
            
            avg_cases = num_cases/323
            
            county_data.at[idx, "CASES"] = num_cases
            county_data.at[idx, "DEATHS"] = num_deaths
            county_data.at[idx, "TOT_POP"] = tot_pop
            county_data.at[idx, "AVG_CASES_PER_DAY"] = avg_cases
            county_data.at[idx, "DEATH_RATE"] = num_deaths/num_cases if num_cases != 0 else 0
            county_data.at[idx, "TOT_MALE"] = male_pop
            county_data.at[idx, "TOT_FEMALE"] = fem_pop
            county_data.at[idx, "POVALL_2018"] = poor
            county_data.at[idx, "MED_INC_STATE_TOTAL"] = med
            county_data.at[idx, "LESS_THAN_HSD"] = less_than_hsd
            county_data.at[idx, "ONLY_HSD"] = only_hsd
            county_data.at[idx, "BCH_OR_HIGHER"] = bch_or_higher
    
    return county_data

In [35]:
df = add_features(county_data, missing, covid_cases, covid_deaths, population, unemployment, education, poverty)

In [36]:
df.drop(columns="JHU_FIPS", inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(2984, 85)

In [37]:
%%R -i df -w 18 -h 15 --units in -r 500

options(warn=-1)

suppressMessages(library(corrplot))
suppressMessages(library(ggplot2))
suppressMessages(library(ggcorrplot))
suppressMessages(library(ggiraph))
suppressMessages(library(ggiraphExtra))
suppressMessages(library(dplyr))

options(scipen = 999)

data <- data.frame(df)
data <- select_if(data, is.numeric)

r <- cor(data)
round(r,2)
ggcorrplot(r, hc.order = FALSE, 
           lab = FALSE, 
           lab_size = 3,
           tl.cex = 8,
           tl.srt = 90) + 
  theme(plot.title = element_text(angle=90, vjust = 0.10, hjust = 0.10, size=10))

In [38]:
df.drop(columns=["POPEST_2018_B_TOTAL", "POPEST_2018_B_MEDAGEYRS", "PER_ADULT_SMOKING", "PER_18TO64_SMOKING", "PER_65PLUS_SMOKING", "VETPOP_2019_65TO84", 
                 "VETPOP_2019_85PLUS", "VETPOP_2019_TOTAL",  "PER_VET_85PLUS", "PER_VET_65PLUS", "PER_VET_ADULT", "PER_UNINSURED", "PER_CHILD", "PER_ADULT", "ICU_BEDS", 
                 "STAFFED_BEDS", "PER_65PLUS_SMOKING", "PER_18TO64_SMOKING", "SIZE_OWNER", "SIZE_RENTER", "AREA_TOTAL", "AREA_WATER", "VOTE_DEM", "VOTE_GOP", "VOTE_TOTAL", 
                 "PER_DEM", "PER_GOP", "AIR_QUAL_PM2.5", "PER_RURAL", "PER_BEDS", "POP_RURAL", "NUM_PHYS", "PHYS_RATIO", "PREVENT_RATE", "PER_VACCINATED", "PER_UNINSURED", 
                 "DENSITY_POP", "LICENSED_BEDS"], inplace=True)

In [39]:
def correlation(df, threshold=0.8):
    states = df["STATE_ABBR"].unique().tolist()
    corr = {}
    # for every state in the U.S
    for state in states:
        # find the number of cases in that state
        # cases variable is a dataframe with COVID-19 cases
        # for each county in the state
        cases = df.loc[df["STATE_ABBR"] == state]["CASES"]
        # calculate correlation for each feature with the
        # number of COVID-19 cases in the state
        corr_dict = df.loc[df["STATE_ABBR"] == state].corrwith(cases).to_dict()
        # sort the dictionary so that higher correlation is closer to the left
        corr_dict = {k: v for k, v in sorted(corr_dict.items(), key=lambda item: item[1], reverse=True)}
        # get keys for high correlation values
        corr_feat = [k for k, v in corr_dict.items() if v >= threshold and (k != "CASES" and k != "AVG_CASES_PER_DAY")]
        # create a dictionary where only the high 
        # correlation values are listed, and low
        # correlation values are dropped from analysis
        corr_dict = {k: corr_dict[k] for k in corr_feat}
        # assign the corr_dict dictionary to the 
        # corr dictionary with the state serving
        # as the key
        corr[state] = corr_dict
    return corr

In [40]:
rho = correlation(df)

#### U.S.

In [41]:
%%R -i df -w 15 -h 10 --units in -r 500

data <- data.frame(df)

print(mean(data$CASES))

CASES <- data$CASES
AFRICAN_AMERICAN <- data$POP_BLACK
CAUCASIAN <- data$POP_WHITE
HISPANIC <- data$POP_HIS
ASIAN <- data$POP_ASIAN

fit = lm(CASES ~ AFRICAN_AMERICAN + CAUCASIAN + HISPANIC + ASIAN)
summary(fit)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####
            #### ONE FACTOR ONLY ggplot ####
            #### TWO FACTORS FOR ggPredict ####

# ggplot(data,aes(y=CASES,x=UNDER_18))+geom_point()+geom_smooth(method="lm")
# ggPredict(fit,se=TRUE,interactive=TRUE)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####

[1] 4832.74

Call:
lm(formula = CASES ~ AFRICAN_AMERICAN + CAUCASIAN + HISPANIC + 
    ASIAN)

Residuals:
   Min     1Q Median     3Q    Max 
-57944   -350     14    399  78563 

Coefficients:
                   Estimate Std. Error t value            Pr(>|t|)    
(Intercept)      19.0385949 78.2623487   0.243               0.808    
AFRICAN_AMERICAN  0.0715340  0.0019217  37.225 <0.0000000000000002 ***
CAUCASIAN         0.0434228  0.0007935  54.720 <0.0000000000000002 ***
HISPANIC          0.0752060  0.0009577  78.530 <0.0000000000000002 ***
ASIAN            -0.0490961  0.0028394 -17.291 <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3815 on 2979 degrees of freedom
Multiple R-squared:  0.9472,	Adjusted R-squared:  0.9471 
F-statistic: 1.336e+04 on 4 and 2979 DF,  p-value: < 0.00000000000000022



**NOTE**

Above reflects mostly everyone in the U.S. or in any other diverse country for that matter. Other factors are perhaps better studied. Also there is a lot of features and the factors can be studied in many different combinations to build a model, so a handful of States are selected. There are two types of States to consider:

    1. Those hit hardest in economic terms 
            https://www.weforum.org/agenda/2020/10/covid19-coronavirus-pandemic-economics/
            
    2. Those with the highest number of COVID-19 cases per 100,000
            https://www.statista.com/statistics/1109004/coronavirus-covid19-cases-rate-us-americans-by-state/

#### Alabama

In [42]:
alabama = df.loc[df["STATE_ABBR"] == "AL"]

In Alabama, poverty amongst caucasian residence better explains the data (96.66%) than poverty amongst african american residents (92.85%). However, if we look past the race we notice that unemployment amongst female residents explains 97% of the data in Alabama, which is higher either of previous criterias.

In [43]:
%%R -i alabama -w 15 -h 10 --units in -r 500

data <- data.frame(alabama)

print(mean(data$CASES))

CASES <- data$CASES
UNEMPLOYED <- data$POVALL_2018
FEMALE <- data$TOT_FEMALE

fit = lm(CASES ~ UNEMPLOYED + FEMALE)
summary(fit)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####
            #### ONE FACTOR ONLY ggplot ####
            #### TWO FACTORS FOR ggPredict ####

# ggplot(data,aes(y=CASES,x=UNDER_18))+geom_point()+geom_smooth(method="lm")
# ggPredict(fit,se=TRUE,interactive=TRUE)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####

[1] 4181.896

Call:
lm(formula = CASES ~ UNEMPLOYED + FEMALE)

Residuals:
    Min      1Q  Median      3Q     Max 
-3322.9  -350.7  -119.3   185.3  3134.1 

Coefficients:
              Estimate Std. Error t value            Pr(>|t|)    
(Intercept) 213.086522 134.382641   1.586              0.1177    
UNEMPLOYED    0.046053   0.025137   1.832              0.0716 .  
FEMALE        0.090711   0.007635  11.881 <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 897 on 64 degrees of freedom
Multiple R-squared:  0.9776,	Adjusted R-squared:  0.9769 
F-statistic:  1397 on 2 and 64 DF,  p-value: < 0.00000000000000022



#### North Dakota

In [44]:
north_dakota = df.loc[df["STATE_ABBR"] == "ND"]

In North Dakota male African American residents explained 99% of the data. If socio-economic factors of Afrian-American male residents in North Dakota are studied perhaps they may apply to other residents too and help explain the data. North Dakota ranks first when it comes to number of cases per 100,000 residents.

In [45]:
%%R -i north_dakota -w 15 -h 10 --units in -r 500

data <- data.frame(north_dakota)

print(mean(data$CASES))

CASES <- data$CASES
AFRICAN_AMERICAN <- data$POP_BLACK
MALE <- data$TOT_MALE

fit = lm(CASES ~ MALE + AFRICAN_AMERICAN)
summary(fit)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####
            #### ONE FACTOR ONLY ggplot ####
            #### TWO FACTORS FOR ggPredict ####
    
# suppressMessages(ggplot(data,aes(y=CASES,x=AFRICAN_AMERICAN))+geom_point()+geom_smooth(method="lm"))
# ggPredict(fit,se=TRUE,interactive=TRUE)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####

[1] 1625.453

Call:
lm(formula = CASES ~ MALE + AFRICAN_AMERICAN)

Residuals:
    Min      1Q  Median      3Q     Max 
-859.78  -99.09   61.61  125.85  634.17 

Coefficients:
                    Estimate  Std. Error t value             Pr(>|t|)    
(Intercept)      -225.695809   44.015906  -5.128  0.00000477712235431 ***
MALE                0.302608    0.007589  39.875 < 0.0000000000000002 ***
AFRICAN_AMERICAN   -0.784372    0.068485 -11.453  0.00000000000000138 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 252.5 on 50 degrees of freedom
Multiple R-squared:  0.9945,	Adjusted R-squared:  0.9943 
F-statistic:  4524 on 2 and 50 DF,  p-value: < 0.00000000000000022



#### Arizona

In [46]:
arizona = df.loc[df["STATE_ABBR"] == "AZ"]

In [47]:
%%R -i arizona -w 15 -h 10 --units in -r 500

data <- data.frame(arizona)

print(mean(data$CASES))

CASES <- data$CASES
AGE_UNDER_18 <- data$POPEST_2018_B_U18
EMPLOYMENT <- data$UNEMPLOYED

fit = lm(CASES ~ AGE_UNDER_18 + EMPLOYMENT)
summary(fit)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####
            #### ONE FACTOR ONLY ggplot ####
            #### TWO FACTORS FOR ggPredict ####
    
# suppressMessages(ggplot(data,aes(y=CASES,x=AFRICAN_AMERICAN))+geom_point()+geom_smooth(method="lm"))
# ggPredict(fit,se=TRUE,interactive=TRUE)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####

[1] 25506.73

Call:
lm(formula = CASES ~ AGE_UNDER_18 + EMPLOYMENT)

Residuals:
    Min      1Q  Median      3Q     Max 
-3192.1 -1304.0   556.3  1107.7  2440.1 

Coefficients:
               Estimate Std. Error t value     Pr(>|t|)    
(Intercept)  -357.91927  590.33133  -0.606      0.55561    
AGE_UNDER_18    0.16674    0.01463  11.398 0.0000000856 ***
EMPLOYMENT      0.68636    0.16729   4.103      0.00146 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1880 on 12 degrees of freedom
Multiple R-squared:  0.9992,	Adjusted R-squared:  0.999 
F-statistic:  7209 on 2 and 12 DF,  p-value: < 0.00000000000000022



In [48]:
mean_under_18 = round(arizona["POPEST_2018_B_U18"].mean(), 0)
print(f"Average number of people under the age of 18: {mean_under_18}")

Average number of people under the age of 18: 109510.0


In [49]:
mean_unemployed = round(arizona["UNEMPLOYED"].mean(), 0)
print(f"Average number of unemployed people: {mean_unemployed}")

Average number of unemployed people: 11080.0


The Latest: Young people most of Arizona’s confirmed cases

Reference: https://apnews.com/article/f8101d9027c56e8250a5658c19e99bf3

#### Kansas

In [50]:
kansas = df.loc[df["STATE_ABBR"] == "KS"]

In [51]:
%%R -i kansas -w 15 -h 10 --units in -r 500

data <- data.frame(kansas)

print(mean(data$CASES))

CASES <- data$CASES
AGE_UNDER_18 <- data$POPEST_2018_B_U18
POVERTY <- data$POVALL_2018
EMPLOYMENT <- data$UNEMPLOYED

fit = lm(CASES ~ AGE_UNDER_18 + POVERTY + EMPLOYMENT)
summary(fit)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####
            #### ONE FACTOR ONLY ggplot ####
            #### TWO FACTORS FOR ggPredict ####
    
# suppressMessages(ggplot(data,aes(y=CASES,x=AFRICAN_AMERICAN))+geom_point()+geom_smooth(method="lm"))
# ggPredict(fit,se=TRUE,interactive=TRUE)

##### UNCOMMENT THIS FOR PREDICTION GRAPHS #####

[1] 1741.457

Call:
lm(formula = CASES ~ AGE_UNDER_18 + POVERTY + EMPLOYMENT)

Residuals:
     Min       1Q   Median       3Q      Max 
-1438.68  -162.74   -85.72    30.56  2594.60 

Coefficients:
              Estimate Std. Error t value         Pr(>|t|)    
(Intercept)  116.99713   55.26661   2.117         0.036720 *  
AGE_UNDER_18   0.36507    0.04683   7.795 0.00000000000595 ***
POVERTY        0.16232    0.02302   7.050 0.00000000022651 ***
EMPLOYMENT    -2.84624    0.76282  -3.731         0.000315 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 523.4 on 101 degrees of freedom
Multiple R-squared:  0.9872,	Adjusted R-squared:  0.9868 
F-statistic:  2601 on 3 and 101 DF,  p-value: < 0.00000000000000022

